In [9]:
import Content_Based_Filter,Hyper_parameter_tuning
import pandas as pd


In [10]:
%load_ext autoreload

In [11]:
%autoreload 2

In [2]:
df=pd.read_csv('/home/gm/Desktop/ExcelR_Projects/book_recommendation/preprocessing_cleaning/book_details_bart.csv')

In [13]:
tuner=Hyper_parameter_tuning.HyperparameterTuner(df)


In [14]:
%autoreload 2

In [3]:
df=df[~df.book_details.isna()]

In [20]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import boxcox
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import re

class cleaning_dirty_data():

    def __init__(self):
        

        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()
        # print('loaded cleaning_data class')

    def text_cleaning(self,description):
            # Removing Emojis
            desc=self.__remove_emojis(description)
            
            # Lowercase the text
            text = desc.lower()

            # Remove newlines
            text = text.replace('\n', ' ')
            
            # Tokenize the text
            tokens = word_tokenize(text)
            
            # Remove punctuation and special characters and numbers
            tokens = [word for word in tokens if word.isalpha()]
            
            # Remove stopwords
            tokens = [word for word in tokens if word not in self.stop_words]
            
            # Lemmatize the tokens
            tokens = [self.lemmatizer.lemmatize(word) for word in tokens]

            
            return tokens
    
    def __remove_emojis(self,text):
        """
        Remove emojis from the given text.

        Parameters:
            text (str): The text from which emojis need to be removed.

        Returns:
            str: The text with emojis removed.
        """
        emoji_pattern = re.compile(
                "["
                "\U0001F600-\U0001F64F"  # emoticons
                "\U0001F300-\U0001F5FF"  # symbols & pictographs
                "\U0001F680-\U0001F6FF"  # transport & map symbols
                "\U0001F1E0-\U0001F1FF"  # flags (iOS)
                "\U00002702-\U000027B0"  # dingbats
                "\U000024C2-\U0001F251" 
                "]+", 
                flags=re.UNICODE
            )            
        return emoji_pattern.sub(r'', text)

In [98]:

class cleaning_dirty_data():

    def __init__(self):
        

        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()
        # print('loaded cleaning_data class')

    def text_cleaning(self,description):
            
            # Removing Emojis
            desc=self.__remove_emojis(description)
            
            # Lowercase the text
            text = desc.lower()

            # Remove newlines
            text = text.replace('\n', ' ')
            
            # Tokenize the text
            # tokens = word_tokenize(text)
            
            # Remove punctuation and special characters and numbers
            # text = text.isalnum()
            
            # # Lemmatize the tokens
            # tokens = [self.lemmatizer.lemmatize(word) for word in tokens]

            
            return text
    
    def __remove_emojis(self,text):
        """
        Remove emojis from the given text.

        Parameters:
            text (str): The text from which emojis need to be removed.

        Returns:
            str: The text with emojis removed.
        """
        emoji_pattern = re.compile(
                "["
                "\U0001F600-\U0001F64F"  # emoticons
                "\U0001F300-\U0001F5FF"  # symbols & pictographs
                "\U0001F680-\U0001F6FF"  # transport & map symbols
                "\U0001F1E0-\U0001F1FF"  # flags (iOS)
                "\U00002702-\U000027B0"  # dingbats
                "\U000024C2-\U0001F251" 
                "]+", 
                flags=re.UNICODE
            )            
        return emoji_pattern.sub(r'', text)

In [171]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import ParameterGrid
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from nltk.tokenize import word_tokenize
import re
# from preprocessing_cleaning import cleaning_methods
preprocessing_data=cleaning_dirty_data()

class ContentBasedRecommender:
    def __init__(self, book_data, word2vec_params=None, doc2vec_params=None):
        self.book_data = book_data
        self.word2vec_params = word2vec_params if word2vec_params else {'vector_size': 100, 'window': 5, 'min_count': 1, 'workers': 4}
        self.doc2vec_params = doc2vec_params if doc2vec_params else {'vector_size': 100, 'min_count': 1, 'epochs': 10}

    def preprocessing(self, text):
        return preprocessing_data.text_cleaning(text)


    def generate_embeddings(self):

        def preprocess_text(col):
            for i in col:
                self.book_data[f'{i}_tokens'] = self.book_data[f'{i}'].apply(self.preprocessing)
        
        col=['book_title','Genre_Interpretation','author','book_details']
        preprocess_text(col)
            

        # self.book_data['book_title_tokens'] = self.book_data['book_title'].apply(self.preprocessing)
        # self.book_data['genres_tokens'] = self.book_data['Genre_Interpretation'].apply(self.preprocessing)
        # self.book_data['author_tokens'] = self.book_data['author'].apply(self.preprocessing)
        # self.book_data['book_details_tokens'] = self.book_data['book_details'].apply(self.preprocessing)

        # Train Word2Vec model on combined text data from titles, genres, and authors
        all_tokens = self.book_data['book_title_tokens'].tolist() + self.book_data['genres_tokens'].tolist() + self.book_data['author_tokens'].tolist()
        
        word2vec_model = Word2Vec(sentences=all_tokens, **self.word2vec_params)

        # Generate Word2Vec embeddings for titles, genres, and authors
        self.book_data['title_embedding'] = self.book_data['book_title_tokens'].apply(lambda tokens: np.mean([word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv], axis=0))
        self.book_data['genres_embedding'] = self.book_data['genres_tokens'].apply(lambda tokens: np.mean([word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv], axis=0))
        self.book_data['author_embedding'] = self.book_data['author_tokens'].apply(lambda tokens: np.mean([word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv], axis=0))

        # Tagging documents for Doc2Vec
        tagged_data = [TaggedDocument(words=row['book_details_tokens'], tags=[str(row['book_id'])]) for index, row in self.book_data.iterrows()]

        # Train Doc2Vec model
        doc2vec_model = Doc2Vec(**self.doc2vec_params)
        doc2vec_model.build_vocab(tagged_data)
        doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

        # Generate Doc2Vec embeddings for book details
        self.book_data['details_embedding'] = self.book_data['book_id'].apply(lambda id: doc2vec_model.dv[str(id)])

        # Combine embeddings
        def combine_embeddings(row):
            title_emb = row['title_embedding']
            genres_emb = row['genres_embedding']
            author_emb = row['author_embedding']
            details_emb = row['details_embedding']
            
            combined_emb = np.concatenate((title_emb, genres_emb, author_emb, details_emb))
            return combined_emb

        self.book_data['combined_embedding'] = self.book_data.apply(combine_embeddings, axis=1)

        # Calculate cosine similarities between book embeddings
        combined_embeddings_matrix = np.stack(self.book_data['combined_embedding'].values)
        cosine_similarities = cosine_similarity(combined_embeddings_matrix, combined_embeddings_matrix)
        return cosine_similarities

    def get_recommendations_for_book(self, book_title='', top_n=5):
        # Find the index corresponding to the given book title
        book_index = self.book_data[self.book_data['book_title'] == book_title].index[0]
        cosine_similarities = self.generate_embeddings()
        recommended_books = self.get_recommendations(book_index, cosine_similarities, top_n)
        return recommended_books

    def get_recommendations(self, book_index, cosine_similarities, top_n=5):
        sim_scores = list(enumerate(cosine_similarities[book_index]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:top_n+1]  # Exclude the book itself
        recommended_books = [(self.book_data.iloc[i]['book_title'], sim_scores[i][1]) for i in range(len(sim_scores))]
        return recommended_books

    def tune_doc2vec(self, param_grid):
        best_params = None
        best_score = float('-inf')

        for params in ParameterGrid(param_grid):
            print(f'Training Doc2Vec with params: {params}')
            # Preprocess the relevant columns
            self.book_data['book_details_tokens'] = self.book_data['book_details'].apply(self.preprocessing)

            tagged_data = [TaggedDocument(words=row['book_details_tokens'], tags=[str(row['book_id'])]) for index, row in self.book_data.iterrows()]
            model = Doc2Vec(**params)
            model.build_vocab(tagged_data)
            model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

            # Evaluate the model
            score = self.evaluate_model_doc(model)
            if score > best_score:
                best_score = score
                best_params = params

        return best_params, best_score

    def evaluate_model_doc(self, model):
        # Retrieve the document vectors for each book ID in the dataset
        book_embeddings = [model.dv[str(book_id)] for book_id in self.book_data['book_id']]
        
        # Compute the cosine similarity matrix for all book embeddings
        cosine_similarities = cosine_similarity(book_embeddings)
        
        # Set the diagonal values of the similarity matrix to 0 to ignore self-similarity
        np.fill_diagonal(cosine_similarities, 0)
        
        # Compute the average cosine similarity value
        average_similarity = cosine_similarities.mean()
        
        return average_similarity
    
    def evaluate_model_word(self, model):
        # Retrieve the document vectors for each book ID in the dataset
        book_embeddings = [model.wv[str(book_id)] for book_id in self.book_data['book_id']]
        
        # Compute the cosine similarity matrix for all book embeddings
        cosine_similarities = cosine_similarity(book_embeddings)
        
        # Set the diagonal values of the similarity matrix to 0 to ignore self-similarity
        np.fill_diagonal(cosine_similarities, 0)
        
        # Compute the average cosine similarity value
        average_similarity = cosine_similarities.mean()
        
        return average_similarity
    

    def tune_word2vec(self, param_grid):
        best_params = None
        best_score = float('-inf')

        for params in ParameterGrid(param_grid):
            print(f'Training Word2Vec with params: {params}')
            # Preprocess the relevant columns
            self.book_data['book_title_tokens'] = self.book_data['book_title'].apply(self.preprocessing)
            self.book_data['genres_tokens'] = self.book_data['Genre_Interpretation'].apply(self.preprocessing)
            self.book_data['author_tokens'] = self.book_data['author'].apply(self.preprocessing)

            all_tokens = self.book_data['book_title_tokens'].tolist() + self.book_data['genres_tokens'].tolist() + self.book_data['author_tokens'].tolist()
            model = Word2Vec(sentences=all_tokens, **params)
            return model

            # Evaluate the model on some metric (e.g., cosine similarity)
            score = self.evaluate_model_word(model)
            if score > best_score:
                best_score = score
                best_params = params

        return best_params, best_score



# # Tune Doc2Vec parameters
# param_grid = {'vector_size': [50, 100], 'min_count': [1, 2], 'epochs': [10, 20]}
# best_params, best_score = recommender.tune_doc2vec(param_grid)
# print('Best Doc2Vec Params:', best_params)
# print('Best Score:', best_score)


In [154]:
import re
import string
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import ParameterGrid

class cleaning_dirty_data:
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()

    def __remove_emojis(self, text):
        emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F700-\U0001F77F"  # alchemical symbols
                                   u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                                   u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                                   u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                                   u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                                   u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                                   "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', text)

    def text_cleaning(self, text):
        # Remove emojis
        text = self.__remove_emojis(text)

        # Lowercase the text
        text = text.lower()

        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))

        # Remove numbers
        text = re.sub(r'\d+', '', text)

        # Tokenize the text
        tokens = word_tokenize(text)

        # Remove stop words and non-alphabetic tokens
        tokens = [word for word in tokens if word.isalpha() and word not in self.stop_words]

        # Lemmatize the tokens
        tokens = [self.lemmatizer.lemmatize(word) for word in tokens]

        return tokens

preprocessing_data = cleaning_dirty_data()

class ContentBasedRecommender:
    def __init__(self, book_data, word2vec_params=None, doc2vec_params=None):
        self.book_data = book_data
        self.word2vec_params = word2vec_params if word2vec_params else {'vector_size': 100, 'window': 5, 'min_count': 1, 'workers': 4}
        self.doc2vec_params = doc2vec_params if doc2vec_params else {'vector_size': 100, 'min_count': 1, 'epochs': 10}

    def preprocessing(self, text):
        return preprocessing_data.text_cleaning(text)

    def pad_or_truncate(self, tokens, max_length, pad_token="<PAD>"):
        if len(tokens) > max_length:
            return tokens[:max_length]
        else:
            return tokens + [pad_token] * (max_length - len(tokens))

    def generate_embeddings(self):
        col = ['book_title', 'genre', 'author', 'book_details']
        for column in col:
            self.book_data[f'{column}_tokens'] = self.book_data[column].apply(self.preprocessing)

        # Determine fixed length for padding/truncation
        lengths = [self.book_data[f'{column}_tokens'].apply(len) for column in col]
        fixed_length = int(np.percentile(np.concatenate(lengths), 90))
        print(f"Fixed length for padding/truncation: {fixed_length}")

        for column in col:
            self.book_data[f'{column}_tokens'] = self.book_data[f'{column}_tokens'].apply(lambda tokens: self.pad_or_truncate(tokens, fixed_length))

        all_tokens = self.book_data['book_title_tokens'].tolist() + self.book_data['genres_tokens'].tolist() + self.book_data['author_tokens'].tolist()
        
        word2vec_model = Word2Vec(sentences=all_tokens, **self.word2vec_params)

        # Generate Word2Vec embeddings for titles, genres, and authors
        self.book_data['title_embedding'] = self.book_data['book_title_tokens'].apply(lambda tokens: np.mean([word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv], axis=0))
        self.book_data['genres_embedding'] = self.book_data['genres_tokens'].apply(lambda tokens: np.mean([word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv], axis=0))
        self.book_data['author_embedding'] = self.book_data['author_tokens'].apply(lambda tokens: np.mean([word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv], axis=0))

        # Tagging documents for Doc2Vec
        tagged_data = [TaggedDocument(words=row['book_details_tokens'], tags=[str(row['book_id'])]) for index, row in self.book_data.iterrows()]

        # Train Doc2Vec model
        doc2vec_model = Doc2Vec(**self.doc2vec_params)
        doc2vec_model.build_vocab(tagged_data)
        doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

        # Generate Doc2Vec embeddings for book details
        self.book_data['details_embedding'] = self.book_data['book_id'].apply(lambda id: doc2vec_model.dv[str(id)])

        # Combine embeddings
        def combine_embeddings(row):
            title_emb = row['title_embedding']
            genres_emb = row['genres_embedding']
            author_emb = row['author_embedding']
            details_emb = row['details_embedding']
            combined_emb = np.concatenate((title_emb, genres_emb, author_emb, details_emb))
            return combined_emb

        self.book_data['combined_embedding'] = self.book_data.apply(combine_embeddings, axis=1)

        # Calculate cosine similarities between book embeddings
        combined_embeddings_matrix = np.stack(self.book_data['combined_embedding'].values)
        cosine_similarities = cosine_similarity(combined_embeddings_matrix, combined_embeddings_matrix)
        return cosine_similarities

    # def get_recommendations(self, book_index, cosine_similarities, top_n=5):
    #     sim_scores = list(enumerate(cosine_similarities[book_index]))
    #     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    #     sim_scores = sim_scores[1:top_n+1]  # Exclude the book itself
    #     book_indices = [i[0] for i in sim_scores]
    #     recommended_books = self.book_data.iloc[book_indices][['book_id', 'book_title']]
    #     return recommended_books

    def get_recommendations_for_book(self, book_title='', top_n=5):
        # Find the index corresponding to the given book title
        book_index = self.book_data[self.book_data['book_title'] == book_title].index[0]
        cosine_similarities = self.generate_embeddings()
        recommended_books = self.get_recommendations(book_index, cosine_similarities, top_n)
        return recommended_books

    def get_recommendations(self, book_index, cosine_similarities, top_n=5):
        sim_scores = list(enumerate(cosine_similarities[book_index]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:top_n+1]  # Exclude the book itself
        recommended_books = [(self.book_data.iloc[i]['book_title'], sim_scores[i][1]) for i in range(len(sim_scores))]
        return recommended_books

    def tune_doc2vec(self, param_grid):
        best_params = None
        best_score = float('-inf')

        for params in ParameterGrid(param_grid):
            print(f'Training Doc2Vec with params: {params}')
            # Preprocess the relevant columns
            self.book_data['book_details_tokens'] = self.book_data['book_details'].apply(self.preprocessing)

            tagged_data = [TaggedDocument(words=row['book_details_tokens'], tags=[str(row['book_id'])]) for index, row in self.book_data.iterrows()]
            # return tagged_data
            model = Doc2Vec(**params)
            model.build_vocab(tagged_data)
            model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

            # Evaluate the model
            return model
            score = self.evaluate_model_doc(model)
            if score > best_score:
                best_score = score
                best_params = params

        return best_params, best_score

    def evaluate_model_doc(self, model):
        # Retrieve the document vectors for each book ID in the dataset
        book_embeddings = [model.dv[str(book_id)] for book_id in self.book_data['book_id']]
        
        # Compute the cosine similarity matrix for all book embeddings
        cosine_similarities = cosine_similarity(book_embeddings)
        
        # Set the diagonal values of the similarity matrix to 0 to ignore self-similarity
        np.fill_diagonal(cosine_similarities, 0)
        
        # Compute the average cosine similarity value
        average_similarity = cosine_similarities.mean()
        
        return average_similarity
    
    def evaluate_model_word(self, model):
        # Retrieve the document vectors for each book ID in the dataset
        book_embeddings = [model.wv[str(book_id)] for book_id in self.book_data['book_id']]
        
        # Compute the cosine similarity matrix for all book embeddings
        cosine_similarities = cosine_similarity(book_embeddings)
        
        # Set the diagonal values of the similarity matrix to 0 to ignore self-similarity
        np.fill_diagonal(cosine_similarities, 0)
        
        # Compute the average cosine similarity value
        average_similarity = cosine_similarities.mean()
        
        return average_similarity
    

    def tune_word2vec(self, param_grid):
        best_params = None
        best_score = float('-inf')

        for params in ParameterGrid(param_grid):
            print(f'Training Word2Vec with params: {params}')
            # Preprocess the relevant columns
            self.book_data['book_title_tokens'] = self.book_data['book_title'].apply(self.preprocessing)
            self.book_data['genres_tokens'] = self.book_data['genres'].apply(self.preprocessing)
            self.book_data['author_tokens'] = self.book_data['author'].apply(self.preprocessing)

            all_tokens = self.book_data['book_title_tokens'].tolist() + self.book_data['genres_tokens'].tolist() + self.book_data['author_tokens'].tolist()
            #CHECK
            # return all_tokens
            model = Word2Vec(sentences=all_tokens, **params)
            # return model

            # Evaluate the model on some metric (e.g., cosine similarity)
            score = self.evaluate_model_word(model)
            if score > best_score:
                best_score = score
                best_params = params

        return best_params, best_score



In [155]:
recommender = ContentBasedRecommender(df)
model=recommender.tune_doc2vec(param_grid)

Training Doc2Vec with params: {'epochs': 10, 'min_count': 1, 'vector_size': 100}


In [161]:
for i in range(100):
    print(model[i].tags)

['57094644']
['895185']
['2948832']
['154126']
['298663']
['53288434']
['359375']
['8752457']
['186439']
['292455']
['132464']
['20758104']
['1932695']
['17333230']
['696754']
['19001']
['15764']
['14461']
['12063467']
['1158967']
['4162673']
['7697361']
['104089']
['163165']
['100365']
['1556093']
['3391271']
['42143']
['53052855']
['17312201']
['444304']
['14029']
['6547258']
['363711']
['6538061']
['113436']
['64481']
['4241335']
['184350']
['11387392']
['11441']
['13170596']
['13099738']
['40580']
['9415951']
['377895']
['197514']
['383206']
['59780']
['23566909']
['219813']
['2314553']
['42692']
['485815']
['9542439']
['240130']
['82783']
['395851']
['16071767']
['13183957']
['17792606']
['1090870']
['59947696']
['817139']
['3753652']
['16171291']
['2033352']
['7476122']
['75651263']
['49628']
['3544225']
['103235']
['98935']
['24504922']
['51364']
['32076678']
['205338104']
['3872']
['99951']
['604117']
['380699']
['15811554']
['31365']
['277397']
['19314']
['913689']
['199784']


In [150]:
model.dv[str(8)]

array([ 0.02729687,  0.09226155,  0.02247115, -0.01870333,  0.08487377,
       -0.13346702, -0.00525219,  0.19999187, -0.07704717, -0.1768693 ,
       -0.03866559,  0.0010055 ,  0.02363927, -0.01772976,  0.04241053,
        0.0182705 ,  0.03446885, -0.04759468,  0.04700487, -0.05060288,
       -0.03169983, -0.00693915,  0.0803776 ,  0.00181268,  0.03045673,
       -0.06118424, -0.06863112, -0.0569353 , -0.00349333,  0.05264449,
        0.05544591, -0.03715688,  0.04593103,  0.00840434,  0.01980482,
        0.09085507,  0.00094149, -0.03457462,  0.01214041, -0.04017382,
       -0.02859725, -0.0442252 ,  0.01317042, -0.03673773,  0.10622225,
       -0.10431745, -0.05369131,  0.08258668, -0.03894801,  0.00331493,
        0.05632712,  0.0054237 , -0.03220112, -0.07984757, -0.08236376,
       -0.02391192,  0.03946503, -0.03904434, -0.0595334 , -0.01485441,
        0.02227925,  0.01976136,  0.03337141, -0.02023305, -0.0575042 ,
        0.09383093,  0.01850523,  0.0446913 , -0.05717289,  0.01

In [144]:
recommender = ContentBasedRecommender(df)


recommender = ContentBasedRecommender(df)
param_grid = {'vector_size': [100, 200], 'min_count': [1, 5], 'epochs': [10, 20]}
param_grid_word={'vector_size': [100,200], 'window': [1, 5], 'min_count': [1,5], 'workers': [4]}
best_params, best_score=recommender.tune_word2vec(param_grid_word)
# doc_best_params, doc_best_score = recommender.tune_doc2vec(param_grid)

# model= recommender.tune_word2vec(param_grid_word)

# print('Best Doc2Vec Params:', doc_best_params)
# print('Doc_Best Score:', doc_best_score)

# print('Best params for word2vec:', word_doc_params)`
# print('Best score for word2vec:', word_doc_score)`

Training Doc2Vec with params: {'epochs': 10, 'min_count': 1, 'vector_size': 100}
Training Doc2Vec with params: {'epochs': 10, 'min_count': 1, 'vector_size': 200}


KeyboardInterrupt: 

In [139]:
# book_embeddings = [model.wv[str(book_id)] for book_id in df['book_id']]

In [142]:
model.wv[1],model.wv[str(1)]

KeyError: "Key '1' not present"

In [143]:
vocabulary_keys = model.wv.key_to_index.keys()

# Print all keys
for key in vocabulary_keys:
    print(key)

fiction
fantasy
adult
romance
historical
mystery
young
literature
science
contemporary
classic
thriller
nonfiction
novel
audiobook
paranormal
history
adventure
horror
biography
crime
story
childrens
memoir
war
american
suspense
humor
magic
grade
middle
book
literary
philosophy
short
chick
lit
urban
christian
new
comic
vampire
century
high
politics
th
dystopia
world
religion
graphic
psychology
school
lgbt
help
self
drama
realistic
british
epic
travel
poetry
supernatural
dark
space
spirituality
life
mythology
time
animal
gothic
john
queer
tale
james
magical
france
love
realism
autobiography
military
health
comedy
ii
essay
robert
age
art
teen
detective
mental
david
business
club
picture
personal
fairy
true
opera
apocalyptic
africa
dragon
woman
feminism
michael
action
speculative
coming
king
girl
one
ghost
development
erotica
family
play
death
german
man
america
christianity
william
post
angel
witch
asia
music
sport
night
russia
espionage
inspirational
canada
french
sociology
last
stephen


In [133]:
df['book_id']

0         57094644
1           895185
2          2948832
3           154126
4           298663
           ...    
15566    183057601
15567       337100
15568         9817
15569     32940867
15570         9533
Name: book_id, Length: 15538, dtype: int64

In [116]:
# Get recommendations for the first book
recommendations = recommender.get_recommendations_for_book(book_index=5597, top_n=5)
print(recommendations)

        book_id                      book_title
10179  26892110       The Library at Mount Char
10341  18754855                 The Meat Market
12415    709609   The Gentle Giants of Ganymede
2885   51779181                The Threat Below
9141      35458  Star Wars: Revenge of the Sith


In [103]:
df.iloc[2]

book_id                                                                    2948832
book_title                                                    Seduce Me at Sunrise
book_details                     \nHe'd tried so hard to forget her.\nKev Merri...
format                                                       Mass Market Paperback
author                                                                Lisa Kleypas
num_pages                                                                 0.185632
genres                           ['Historical Romance', 'Romance', 'Historical'...
num_ratings                                                               0.511851
num_reviews                                                               0.524474
average_rating                                                            0.063872
5                                                                         0.555305
4                                                                         0.613293
3   

In [110]:
df.book_title

0          Daughter of the Deep
1                     The Ghost
2          Seduce Me at Sunrise
3        The Discovery of India
4          The Killer Inside Me
                  ...          
15566                      Ours
15567    The Runes of the Earth
15568     Ten Days in the Hills
15569               The Chemist
15570            The Poe Shadow
Name: book_title, Length: 15538, dtype: object

In [111]:
df

,book_id,book_title,book_details,format,author,num_pages,genres,num_ratings,num_reviews,average_rating,...,Genre_Interpretation,book_title_tokens,genres_tokens,author_tokens,book_details_tokens,title_embedding,genres_embedding,author_embedding,details_embedding,combined_embedding
0,57094644,Daughter of the Deep,New York Times #1 best-selling author Rick Rio...,Hardcover,Rick Riordan,0.144748,"['Fantasy', 'Middle Grade', 'Young Adult', 'Sc...",0.254560,0.689025,0.680305,...,Juvenile Fiction,daughter of the deep,juvenile fiction,rick riordan,new york times #1 best-selling author rick rio...,"[0.23547423, -0.58287567, -0.40047345, -0.3619...","[0.31004304, -0.5181925, -0.40233204, -0.16581...","[0.16329542, -0.5854353, -0.31790572, -0.21697...","[-0.0004535695, 0.051589906, 0.23314911, -0.10...","[0.23547423, -0.58287567, -0.40047345, -0.3619..."
1,895185,The Ghost,The stunning new novel from the No. 1 bestsell...,Hardcover,Robert Harris,0.304597,"['Fiction', 'Thriller', 'Mystery', 'Crime', 'P...",0.085572,0.237198,-0.762375,...,Thriller/Mystery,the ghost,thriller/mystery,robert harris,the stunning new novel from the no. 1 bestsell...,"[0.315292, -0.56580627, -0.401836, -0.32642817...","[0.18136747, -0.5509529, -0.10519284, -0.34308...","[0.21403131, -0.61289406, -0.26961514, -0.3762...","[-0.39051175, 0.04000055, 0.07101117, 0.172833...","[0.315292, -0.56580627, -0.401836, -0.32642817..."
2,2948832,Seduce Me at Sunrise,\nHe'd tried so hard to forget her.\nKev Merri...,Mass Market Paperback,Lisa Kleypas,0.185632,"['Historical Romance', 'Romance', 'Historical'...",0.511851,0.524474,0.063872,...,Historical Fiction,seduce me at sunrise,historical fiction,lisa kleypas,he'd tried so hard to forget her. kev merripe...,"[0.2862126, -0.63357484, -0.33792424, -0.36555...","[0.32410988, -0.5967239, -0.38127595, -0.22010...","[0.20291363, -0.6269334, -0.35155484, -0.38032...","[-0.37301525, -0.024865583, 0.14261639, -0.014...","[0.2862126, -0.63357484, -0.33792424, -0.36555..."
3,154126,The Discovery of India,In conjunction with the Jawaharlal Nehru Memor...,Paperback,Jawaharlal Nehru,1.386309,"['History', 'India', 'Nonfiction', 'Politics',...",-0.250787,-0.348547,0.168100,...,Non Fiction,the discovery of india,non fiction,jawaharlal nehru,in conjunction with the jawaharlal nehru memor...,"[0.26313752, -0.61305916, -0.4129222, -0.28184...","[0.31143096, -0.5572727, -0.46147153, -0.13213...","[0.08324257, -0.5531818, -0.44002706, -0.29923...","[-0.16193235, 0.09088329, 0.2791146, -0.120960...","[0.26313752, -0.61305916, -0.4129222, -0.28184..."
4,298663,The Killer Inside Me,"Everyone in the small town of Central City, Te...",Paperback,Jim Thompson,-0.484222,"['Fiction', 'Crime', 'Mystery', 'Noir', 'Thril...",0.207299,0.294163,-0.553099,...,Thriller/Mystery,the killer inside me,thriller/mystery,jim thompson,"everyone in the small town of central city, te...","[0.23803842, -0.65377027, -0.38254514, -0.3489...","[0.18136747, -0.5509529, -0.10519284, -0.34308...","[0.33751965, -0.62454563, -0.3427327, -0.30172...","[-0.062370986, 0.09561469, 0.32022613, -0.0097...","[0.23803842, -0.65377027, -0.38254514, -0.3489..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15566,183057601,Ours,The third and final.Blurb coming soon.,ebook,Julie Mannino,0.023473,"['BDSM', 'Romance', 'M M Romance', 'Fantasy', ...",-2.048186,-1.953341,-0.204847,...,"Supernatural, Mystery, and Romance",ours,"supernatural, mystery, and romance",julie mannino,the third and final.blurb coming soon.,"[0.26091474, -0.62619597, -0.24281347, -0.2997...","[0.19779843, -0.6620997, -0.25377682, -0.46585...","[0.20764875, -0.61332023, -0.44243968, -0.2006...","[-0.031399563, 0.039542645, 0.04523698, -0.050...","[0.26091474, -0.62619597, -0.24281347, -0.2997..."
15567,337100,The Runes of the Earth,Beginning ten years after the end of the accla...,Hardcover,Stephen R. Donaldson,0.946065,"['Fantasy', 'Fiction', 'Epic Fantasy', 'Scienc...",-0.298514,-0

In [114]:
df[df.author=='J.K. Rowling']

,book_id,book_title,book_details,format,author,num_pages,genres,num_ratings,num_reviews,average_rating,...,Genre_Interpretation,book_title_tokens,genres_tokens,author_tokens,book_details_tokens,title_embedding,genres_embedding,author_embedding,details_embedding,combined_embedding
1718,13497818,The Casual Vacancy,A BIG NOVEL ABOUT A SMALL TOWN ...When Barry F...,Hardcover,J.K. Rowling,0.828069,"['Fiction', 'Contemporary', 'Mystery', 'Adult'...",1.600835,1.869436,-1.983578,...,Adult/YA Fiction,the casual vacancy,adult/ya fiction,j.k. rowling,a big novel about a small town ...when barry f...,"[0.2040478, -0.5448256, -0.36187854, -0.273218...","[0.27784055, -0.500711, -0.25795433, -0.221580...","[0.17655687, -0.4983057, -0.4451884, -0.256772...","[-0.22281237, -0.015902221, 0.051603325, -0.05...","[0.2040478, -0.5448256, -0.36187854, -0.273218..."
1942,506583,Harry Potter Schoolbooks: Quidditch Through th...,SHS0545007984 Features: -Subjects: Literacy Ce...,Paperback,J.K. Rowling,0.023473,"['Fantasy', 'Fiction', 'Young Adult', 'Childre...",-0.053942,-0.704460,1.278360,...,Fantasy,harry potter schoolbooks: quidditch through th...,fantasy,j.k. rowling,shs0545007984 features: -subjects: literacy ce...,"[0.24820876, -0.5765957, -0.36998403, -0.29727...","[0.29569158, -0.5232891, -0.36256644, -0.33600...","[0.17655687, -0.4983057, -0.4451884, -0.256772...","[-0.023675602, 0.23625237, -0.014739024, 0.021...","[0.24820876, -0.5765957, -0.36998403, -0.29727..."
2921,2,Harry Potter and the Order of the Phoenix,Harry Potter is about to start his fifth year ...,Paperback,J.K. Rowling,2.143859,"['Young Adult', 'Fiction', 'Magic', 'Childrens...",3.167789,2.284863,1.840247,...,Juvenile Fiction,harry potter and the order of the phoenix,juvenile fiction,j.k. rowling,harry potter is about to start his fifth year ...,"[0.22841614, -0.6169209, -0.35530034, -0.33860...","[0.31004304, -0.5181925, -0.40233204, -0.16581...","[0.17655687, -0.4983057, -0.4451884, -0.256772...","[-0.21589768, 0.082751624, 0.12306019, -0.0856...","[0.22841614, -0.6169209, -0.35530034, -0.33860..."
5248,61302505,Harry Potter and the Cursed Child. Parts One a...,Harry Potter and the Cursed Child has a compl...,Hardcover,J.K. Rowling,0.144748,"['Fantasy', 'Fiction', 'Young Adult', 'Plays',...",2.318627,2.374158,-1.557895,...,Juvenile Fiction,harry potter and the cursed child. parts one a...,juvenile fiction,j.k. rowling,harry potter and the cursed child has a compl...,"[0.21261185, -0.6162914, -0.3480658, -0.337657...","[0.31004304, -0.5181925, -0.40233204, -0.16581...","[0.17655687, -0.4983057, -0.4451884, -0.256772...","[-0.052623037, 0.03652005, 0.06462669, -0.0838...","[0.21261185, -0.6162914, -0.3480658, -0.337657..."
5249,29056083,Harry Potter and the Cursed Child: Parts One a...,Harry Potter and the Cursed Child has a compl...,Hardcover,J.K. Rowling,0.097971,"['Fantasy', 'Fiction', 'Young Adult', 'Plays',...",2.318553,2.374132,-1.557895,...,Juvenile Fiction,harry potter and the cursed child: parts one a...,juvenile fiction,j.k. rowling,harry potter and the cursed child has a compl...,"[0.19169766, -0.59523034, -0.33571118, -0.3126...","[0.31004304, -0.5181925, -0.40233204, -0.16581...","[0.17655687, -0.4983057, -0.4451884, -0.256772...","[-0.16232717, 0.04973044, 0.13651638, -0.02105...","[0.19169766, -0.59523034, -0.33571118, -0.3126..."
5460,1,Harry Potter and the Half-Blood Prince,A relationshippy respite itsSturm und Drangde...,Paperback,J.K. Rowling,1.372944,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...",3.143337,2.224686,2.207087,...,Fantasy,harry potter and the half-blood prince,fantasy,j.k. rowling,a relationshippy respite itssturm und drangde...,"[0.17684716, -0.6065298, -0.35299355, -0.32015...","[0.29569158, -0.5232891, -0.36256644, -0.33600...","[0.17655687, -0.4983057, -0.4451884, -0.256772...","[0.010160258, 0.108383425, 0.056429196, -0.124...","[0.17684716, -0.6065298, -0.35299355, -0.32015..."
5576,8933944,Harry Potter: The Prequel,The Harry Potter 

In [16]:

# def combine_embeddings(row):
#             title_emb = row['title_embedding']
#             genres_emb = row['genres_embedding']
#             author_emb = row['author_embedding']
#             details_emb = row['details_embedding']
            
#             combined_emb = np.concatenate((title_emb, genres_emb, author_emb, details_emb))
#             return combined_emb

#         self.book_data['combined_embedding'] = self.book_data.apply(combine_embeddings, axis=1)


In [35]:
col=['title_embedding','genres_embedding','author_embedding','details_embedding']
for i in col:
    print(len(df[df[i].isna()]))

133
1526
16
0


In [48]:
a=df['book_title'][df['title_embedding'].isna()].values

In [89]:
c=cleaning_dirty_data()

In [90]:
c.text_cleaning(a[0])

'1984'

In [91]:
(a[1])

'After'

In [92]:
a

array(['1984', 'After', 'Forty-Ninth', 'Until You', 'Zom-B', '300',
       'That Was Then, This Is Now', '1984', 'For You', 'The Will',
       'Friend-Zoned', 'Once...', 'Out', 'Stop-Time', 'Why Not-World',
       'Then', 'The Fire-Dwellers', 'Nighty-Nightmare', 'Half-Blood',
       '1Q84', 'Mr.Palomar', 'What Is the What', '1984', 'Good-Bye',
       'What You Did', 'Catch-22', "L'avare", 'Winnie-the-Pooh',
       'The Other', 'Mrs. Piggle-Wiggle', 'NOS4A2', "'Tis", 'I Was Here',
       'Betsy-Tacy', 'Kira-Kira', 'To Have and Have Not', '1984',
       'The Go-Between', 'Where Or When', 'The Fuck-Up', 'The Other',
       'It', '4 3 2 1', 'All of Me', 'R.U.R.', 'Daddy-Long-Legs', '1Q84',
       'Down to You', 'It Had to Be You', 'We3', '2666', 'Number9Dream',
       'Over You', 'Before We Were Yours', 'As Above',
       'Because of Winn-Dixie', 'Slaughterhouse-Five', 'Non-Stop',
       'Babel-17', 'The 100', '2312', 'The Time-Image', '1984', 'Between',
       'So B. It', 'Th1rteen R3ason

In [93]:
for i in a:
    print(c.text_cleaning(i))

1984
after
forty-ninth
until you
zom-b
300
that was then, this is now
1984
for you
the will
friend-zoned
once...
out
stop-time
why not-world
then
the fire-dwellers
nighty-nightmare
half-blood
1q84
mr.palomar
what is the what
1984
good-bye
what you did
catch-22
l'avare
winnie-the-pooh
the other
mrs. piggle-wiggle
nos4a2
'tis
i was here
betsy-tacy
kira-kira
to have and have not
1984
the go-between
where or when
the fuck-up
the other
it
4 3 2 1
all of me
r.u.r.
daddy-long-legs
1q84
down to you
it had to be you
we3
2666
number9dream
over you
before we were yours
as above
because of winn-dixie
slaughterhouse-five
non-stop
babel-17
the 100
2312
the time-image
1984
between
so b. it
th1rteen r3asons why
just for now
fer-de-lance
kick-ass
before
s.
there there
all in
trans-atlantyk
wolf-speaker
just for you
the anti-christ
thr3e
1q84 #1-2
if he had been with me
ex-minnaar
below
you against me
why not me?
th1rt3en
what now?
bel-ami
i'm with you
a
the know-it-all
super-cannes
the ha-ha
the moon-s

In [94]:
c=cleaning_dirty_data()

In [95]:
a=df.author.apply(c.text_cleaning)

In [96]:
a

0                rick riordan
1             robert   harris
2                lisa kleypas
3            jawaharlal nehru
4                jim thompson
                 ...         
15566           julie mannino
15567    stephen r. donaldson
15568             jane smiley
15569         stephenie meyer
15570           matthew pearl
Name: author, Length: 15538, dtype: object

In [97]:
c.text_cleaning(description=list(df.author))

TypeError: expected string or bytes-like object

In [303]:
import re
import string
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import ParameterGrid

class cleaning_dirty_data:
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()

    def __remove_emojis(self, text):
        emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F700-\U0001F77F"  # alchemical symbols
                                   u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                                   u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                                   u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                                   u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                                   u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                                   "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', text)

    def text_cleaning(self, text):
        # Remove emojis
        text = self.__remove_emojis(text)

        # Lowercase the text
        text = text.lower()

        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))

        # Remove numbers
        text = re.sub(r'\d+', '', text)

        # Tokenize the text
        tokens = word_tokenize(text)

        # Remove stop words and non-alphabetic tokens
        tokens = [word for word in tokens if word.isalpha() and word not in self.stop_words]

        # Lemmatize the tokens
        tokens = [self.lemmatizer.lemmatize(word) for word in tokens]

        return tokens

preprocessing_data = cleaning_dirty_data()

class ContentBasedRecommender:
    def __init__(self, book_data, word2vec_params=None, doc2vec_params=None):
        self.book_data = book_data
        self.word2vec_params = word2vec_params if word2vec_params else {'vector_size': 100, 'window': 5, 'min_count': 1, 'workers': 4, 'epochs': 10}
        self.doc2vec_params = doc2vec_params if doc2vec_params else {'vector_size': 100, 'min_count': 1, 'epochs': 10}

    def preprocessing(self, text):
        return preprocessing_data.text_cleaning(text)

    def pad_or_truncate(self, tokens, max_length, pad_token="<PAD>"):
        if len(tokens) > max_length:
            return tokens[:max_length]
        else:
            return tokens + [pad_token] * (max_length - len(tokens))

    def generate_embeddings(self):
        col = ['book_title', 'Genre_Interpretation', 'author', 'book_details']
        for column in col:
            self.book_data[f'{column}_tokens'] = self.book_data[column].apply(self.preprocessing)

        # Determine fixed length for padding/truncation
        lengths = [self.book_data[f'{column}_tokens'].apply(len) for column in col]
        fixed_length = int(np.percentile(np.concatenate(lengths), 90))
        print(f"Fixed length for padding/truncation: {fixed_length}")

        for column in col:
            self.book_data[f'{column}_tokens'] = self.book_data[f'{column}_tokens'].apply(lambda tokens: self.pad_or_truncate(tokens, fixed_length))

        all_tokens = self.book_data['book_title_tokens'].tolist() + self.book_data['genres_tokens'].tolist() + self.book_data['author_tokens'].tolist()
        
        word2vec_model = Word2Vec(sentences=all_tokens, **self.word2vec_params)

        # Generate Word2Vec embeddings for titles, genres, and authors
        self.book_data['title_embedding'] = self.book_data['book_title_tokens'].apply(lambda tokens: np.mean([word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv], axis=0))
        self.book_data['genres_embedding'] = self.book_data['genres_tokens'].apply(lambda tokens: np.mean([word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv], axis=0))
        self.book_data['author_embedding'] = self.book_data['author_tokens'].apply(lambda tokens: np.mean([word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv], axis=0))

        # Tagging documents for Doc2Vec
        tagged_data = [TaggedDocument(words=row['book_details_tokens'], tags=[str(row['book_id'])]) for index, row in self.book_data.iterrows()]

        # Train Doc2Vec model
        doc2vec_model = Doc2Vec(**self.doc2vec_params)
        doc2vec_model.build_vocab(tagged_data)
        doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

        # Generate Doc2Vec embeddings for book details
        self.book_data['details_embedding'] = self.book_data['book_id'].apply(lambda id: doc2vec_model.dv[str(id)])

        # Combine embeddings
        def combine_embeddings(row):
            title_emb = row['title_embedding']
            genres_emb = row['genres_embedding']
            author_emb = row['author_embedding']
            details_emb = row['details_embedding']
            combined_emb = np.concatenate((title_emb, genres_emb, author_emb, details_emb))
            return combined_emb

        self.book_data['combined_embedding'] = self.book_data.apply(combine_embeddings, axis=1)

        # Calculate cosine similarities between book embeddings
        combined_embeddings_matrix = np.stack(self.book_data['combined_embedding'].values)
        cosine_similarities = cosine_similarity(combined_embeddings_matrix, combined_embeddings_matrix)
        return cosine_similarities

    def get_recommendations(self, book_index, cosine_similarities, top_n=5):
        sim_scores = list(enumerate(cosine_similarities[book_index]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:top_n+1]  # Exclude the book itself
        book_indices = [i[0] for i in sim_scores]
        recommended_books = self.book_data.iloc[book_indices][['book_id', 'book_title']]
        return recommended_books

    def get_recommendations_for_book(self, book_index=0, top_n=5):
        cosine_similarities = self.generate_embeddings()
        recommended_books = self.get_recommendations(book_index, cosine_similarities, top_n)
        return recommended_books

    def tune_word2vec(self, param_grid):
        best_params = None
        best_score = float('-inf')

        for params in ParameterGrid(param_grid):
            print(f'Training Word2Vec with params: {params}')
            # Preprocess the relevant columns
            self.book_data['book_title_tokens'] = self.book_data['book_title'].apply(self.preprocessing)
            self.book_data['genres_tokens'] = self.book_data['Genre_Interpretation'].apply(self.preprocessing)
            self.book_data['author_tokens'] = self.book_data['author'].apply(self.preprocessing)

            # Ensure consistent token lengths for concatenation
            all_tokens = self.book_data['book_title_tokens'].tolist() + self.book_data['genres_tokens'].tolist() + self.book_data['author_tokens'].tolist()

            model = Word2Vec(sentences=all_tokens, **params)
            # return model

            # Evaluate the model on some metric (e.g., cosine similarity)
            score = self.evaluate_model_word(model)
            if score > best_score:
                best_score = score
                best_params = params

        return best_params, best_score

    def tune_doc2vec(self, param_grid):
        best_params = None
        best_score = float('-inf')

        for params in ParameterGrid(param_grid):
            print(f'Training Doc2Vec with params: {params}')
            # Preprocess the relevant columns
            self.book_data['book_details_tokens'] = self.book_data['book_details'].apply(self.preprocessing)

            tagged_data = [TaggedDocument(words=row['book_details_tokens'], tags=[str(row['book_id'])]) for index, row in self.book_data.iterrows()]
            
            model = Doc2Vec(**params)
            model.build_vocab(tagged_data)
            model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

            # Evaluate the model
            score = self.evaluate_model_doc(model)
            if score > best_score:
                best_score = score
                best_params = params

        return best_params, best_score

    def evaluate_model_word(self, model):
        # Calculate embeddings for book titles, genres, and authors
        self.book_data['title_embedding'] = self.book_data['book_title_tokens'].apply(lambda tokens: np.mean([model.wv[token] for token in tokens if token in model.wv], axis=0))
        self.book_data['genres_embedding'] = self.book_data['genres_tokens'].apply(lambda tokens: np.mean([model.wv[token] for token in tokens if token in model.wv], axis=0))
        self.book_data['author_embedding'] = self.book_data['author_tokens'].apply(lambda tokens: np.mean([model.wv[token] for token in tokens if token in model.wv], axis=0))

        # Combine embeddings
        def combine_embeddings(row):
            title_emb = row['title_embedding']
            genres_emb = row['genres_embedding']
            author_emb = row['author_embedding']
            # Handle missing or empty embeddings
            if isinstance(title_emb, float) and np.isnan(title_emb):
                title_emb = np.array([])  # Replace NaN with an empty array
            
            if isinstance(genres_emb, float) and np.isnan(genres_emb):
                genres_emb = np.array([])  # Replace NaN with an empty array
            
            if isinstance(author_emb, float) and np.isnan(author_emb):
                author_emb = np.array([])  # Replace NaN with an empty array
            
            
            combined_emb = np.concatenate((title_emb, genres_emb, author_emb))
            return combined_emb

        self.book_data['combined_embedding'] = self.book_data.apply(combine_embeddings, axis=1)
        self.book_data['combined_embedding'] = self.book_data.apply(combine_embeddings, axis=1)

        # Pad or truncate embeddings to a fixed length
        max_length = max(len(embedding) for embedding in self.book_data['combined_embedding'])
        combined_embeddings_matrix = np.stack([np.pad(embedding, (0, max_length - len(embedding))) for embedding in self.book_data['combined_embedding'].values])

        # Compute cosine similarities
        # combined_embeddings_matrix = np.stack(self.book_data['combined_embedding'].values)
        cosine_similarities = cosine_similarity(combined_embeddings_matrix, combined_embeddings_matrix)

        # Set the diagonal values of the similarity matrix to 0 to ignore self-similarity
        np.fill_diagonal(cosine_similarities, 0)

        # Compute the average cosine similarity value
        average_similarity = cosine_similarities.mean()
        return average_similarity

    def evaluate_model_doc(self, model):
        # Generate Doc2Vec embeddings for book details
        self.book_data['details_embedding'] = self.book_data['book_id'].apply(lambda id: model.dv[str(id)])

        # Combine embeddings
        def combine_embeddings(row):
            details_emb = row['details_embedding']
            combined_emb = details_emb  # Assuming you want to evaluate details embedding only
            return combined_emb

        self.book_data['combined_embedding'] = self.book_data.apply(combine_embeddings, axis=1)

        # Compute cosine similarities
        combined_embeddings_matrix = np.stack(self.book_data['combined_embedding'].values)
        cosine_similarities = cosine_similarity(combined_embeddings_matrix, combined_embeddings_matrix)

        # Set the diagonal values of the similarity matrix to 0 to ignore self-similarity
        np.fill_diagonal(cosine_similarities, 0)

        # Compute the average cosine similarity value
        average_similarity = cosine_similarities.mean()
        return average_similarity


In [305]:
class main:
def main():
    df=pd.read_csv('/home/gm/Desktop/ExcelR_Projects/book_recommendation/preprocessing_cleaning/book_details_bart.csv')
    df=df[~df.book_details.isna()]

    cleaning_dirty= cleaning_dirty_data()
    def word_embed_cleaning(df):
        index_to_remove=set()
        cols=['author','book_title','book_details','Genre_Interpretation']
        for c in cols:
            a=df[c].apply(cleaning_dirty.text_cleaning)
            for i,v in a.items():
                if len(v)==0:
                    index_to_remove.add(i)

        df.drop(index_to_remove,inplace=True)
        df.reset_index(inplace=True,drop=True)
        
    word_embed_cleaning(df)


In [312]:
recommender = ContentBasedRecommender(df)
param_grid_doc= {'vector_size': [100, 200], 'min_count': [1, 2, 3], 'epochs': [10, 20]}
param_grid_word={'vector_size': [100,200], 'window': [1, 2, 5], 'min_count': [1], 'workers': [4]}
# model=recommender.tune_word2vec(param_grid_word)
best_param_word, best_score_word=recommender.tune_word2vec(param_grid_word)
best_score_doc,best_score_doc=recommender.tune_doc2vec(param_grid_doc)


Training Word2Vec with params: {'min_count': 1, 'vector_size': 100, 'window': 1, 'workers': 4}
Training Word2Vec with params: {'min_count': 1, 'vector_size': 100, 'window': 2, 'workers': 4}
Training Word2Vec with params: {'min_count': 1, 'vector_size': 100, 'window': 5, 'workers': 4}
Training Word2Vec with params: {'min_count': 1, 'vector_size': 200, 'window': 1, 'workers': 4}
Training Word2Vec with params: {'min_count': 1, 'vector_size': 200, 'window': 2, 'workers': 4}
Training Word2Vec with params: {'min_count': 1, 'vector_size': 200, 'window': 5, 'workers': 4}


In [313]:
print(best_params)


{'min_count': 1, 'vector_size': 200, 'window': 5, 'workers': 4}


In [314]:
print(best_score) 

0.58337045


In [265]:
combined_emb = np.concatenate((df.title_embedding, df.genres_embedding, df.author_embedding))
           

In [281]:
df.loc[3]

book_id                                                                     154126
book_title                                                  The Discovery of India
book_details                     In conjunction with the Jawaharlal Nehru Memor...
format                                                                   Paperback
author                                                            Jawaharlal Nehru
num_pages                                                                 1.386309
genres                           ['History', 'India', 'Nonfiction', 'Politics',...
num_ratings                                                              -0.250787
num_reviews                                                              -0.348547
average_rating                                                              0.1681
5                                                                         -0.07329
4                                                                        -0.100157
3   

In [225]:
class cleaning_dirty_data:
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()

    def __remove_emojis(self, text):
        emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F700-\U0001F77F"  # alchemical symbols
                                   u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                                   u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                                   u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                                   u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                                   u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                                   "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', text)

    def text_cleaning(self, text):
        # Remove emojis
        text = self.__remove_emojis(text)

        # Lowercase the text
        text = text.lower()

        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))

        # Remove numbers
        text = re.sub(r'\d+', '', text)

        # Tokenize the text
        tokens = word_tokenize(text)

        # Remove stop words and non-alphabetic tokens
        tokens = [word for word in tokens if word.isalpha() and word not in self.stop_words]

        # Lemmatize the tokens
        tokens = [self.lemmatizer.lemmatize(word) for word in tokens]

        return tokens


In [246]:

cleaning_dirty= cleaning_dirty_data()
def word_embed_cleaning(df):
    index_to_remove=set()
    cols=['author','book_title','book_details']
    for c in cols:
        a=df[c].apply(cleaning_dirty.text_cleaning)
        for i,v in a.items():
            if len(v)==0:
                index_to_remove.add(i)

    df.drop(index_to_remove,inplace=True)
    df.reset_index(inplace=True,drop=True)


In [247]:
word_embed_cleaning(df)

In [251]:
np.co

In [252]:
df

,index,book_id,book_title,book_details,format,author,num_pages,genres,num_ratings,num_reviews,average_rating,5,4,3,2,1,years_since_first_publication,Genre_Interpretation
0,0,57094644,Daughter of the Deep,New York Times #1 best-selling author Rick Rio...,Hardcover,Rick Riordan,0.144748,"['Fantasy', 'Middle Grade', 'Young Adult', 'Sc...",0.254560,0.689025,0.680305,0.423443,0.375877,0.251464,0.124733,0.003403,-1.894448,Juvenile Fiction
1,1,895185,The Ghost,The stunning new novel from the No. 1 bestsell...,Hardcover,Robert Harris,0.304597,"['Fiction', 'Thriller', 'Mystery', 'Crime', 'P...",0.085572,0.237198,-0.762375,-0.017886,0.289047,0.370226,0.258989,-0.027662,-0.300537,Thriller/Mystery
2,2,2948832,Seduce Me at Sunrise,\nHe'd tried so hard to forget her.\nKev Merri...,Mass Market Paperback,Lisa Kleypas,0.185632,"['Historical Romance', 'Romance', 'Historical'...",0.511851,0.524474,0.063872,0.555305,0.613293,0.595864,0.507648,0.284715,-0.363598,Historical Fiction
3,3,154126,The Discovery of India,In conjunction with the Jawaharlal Nehru Memor...,Paperback,Jawaharlal Nehru,1.386309,"['History', 'India', 'Nonfiction', 'Politics',...",-0.250787,-0.348547,0.168100,-0.073290,-0.100157,-0.176481,-0.178933,0.013985,1.431631,Non Fiction
4,4,298663,The Killer Inside Me,"Everyone in the small town of Central City, Te...",Paperback,Jim Thompson,-0.484222,"['Fiction', 'Crime', 'Mystery', 'Noir', 'Thril...",0.207299,0.294163,-0.553099,0.211275,0.354494,0.387119,0.454474,0.447853,1.338255,Thriller/Mystery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15451,15565,1038645,The Burning Times,The year is 1357. The Inquisition rages throug...,Paperback,Jeanne Kalogridis,0.381049,"['Historical Fiction', 'Fiction', 'Historical'...",-0.936634,-0.855912,-1.310263,-1.020666,-0.880891,-0.661649,-0.517484,-0.616247,0.062579,Historical Fiction
15452,15567,337100,The Runes of the Earth,Beginning ten years after the end of the accla...,Hardcover,Stephen R. Donaldson,0.946065,"['Fantasy', 'Fiction', 'Epic Fantasy', 'Scienc...",-0.298514,-0.463990,-0.762375,-0.269883,-0.148728,-0.034705,-0.002992,0.016072,-0.116297,Fantasy
15453,15568,9817,Ten Days in the Hills,It is the morning after the Academy Awards. Ma...,Hardcover,Jane Smiley,0.606985,"['Fiction', 'Contemporary', 'Romance', 'Litera...",-0.853062,-0.404413,-3.167038,-1.494164,-1.061748,-0.557024,-0.096651,0.296962,-0.248938,Classic Fiction
15454,15569,32940867,The Chemist,"In this gripping page-turner, an ex-agent on t...",Paperback,Stephenie Meyer,0.899438,"['Fiction', 'Thriller', 'Mystery', 'Romance', ...",0.890139,1.229197,-0.849215,0.737328,0.886439,0.991032,1.127764,1.192957,-1.130532,Thriller/Mystery


In [230]:
df['book_title']

0          Daughter of the Deep
1                     The Ghost
2          Seduce Me at Sunrise
3        The Discovery of India
4          The Killer Inside Me
                  ...          
15566                      Ours
15567    The Runes of the Earth
15568     Ten Days in the Hills
15569               The Chemist
15570            The Poe Shadow
Name: book_title, Length: 15538, dtype: object

In [255]:
df

,index,book_id,book_title,book_details,format,author,num_pages,genres,num_ratings,num_reviews,...,1,years_since_first_publication,Genre_Interpretation,book_title_tokens,genres_tokens,author_tokens,title_embedding,genres_embedding,author_embedding,combined_embedding
0,0,57094644,Daughter of the Deep,New York Times #1 best-selling author Rick Rio...,Hardcover,Rick Riordan,0.144748,"['Fantasy', 'Middle Grade', 'Young Adult', 'Sc...",0.254560,0.689025,...,0.003403,-1.894448,Juvenile Fiction,"[daughter, deep]","[juvenile, fiction]","[rick, riordan]","[-0.029608455, 0.040048007, -0.10410403, -0.09...","[-0.009031999, -0.03070344, 0.073201835, 0.136...","[-0.034473933, 0.02165243, -0.08802137, -0.098...","[0.053568132, -0.041865893, -0.003430021, 0.07..."
1,1,895185,The Ghost,The stunning new novel from the No. 1 bestsell...,Hardcover,Robert Harris,0.304597,"['Fiction', 'Thriller', 'Mystery', 'Crime', 'P...",0.085572,0.237198,...,-0.027662,-0.300537,Thriller/Mystery,[ghost],[thrillermystery],"[robert, harris]","[-0.053129148, 0.048214216, -0.12809892, -0.14...","[-0.0087274825, 0.0021301615, -0.0008735442, -...","[-0.059541296, 0.052320734, -0.16741243, -0.17...","[0.07435877, -0.053906452, -0.0075002187, 0.09..."
2,2,2948832,Seduce Me at Sunrise,\nHe'd tried so hard to forget her.\nKev Merri...,Mass Market Paperback,Lisa Kleypas,0.185632,"['Historical Romance', 'Romance', 'Historical'...",0.511851,0.524474,...,0.284715,-0.363598,Historical Fiction,"[seduce, sunrise]","[historical, fiction]","[lisa, kleypas]",NaN,"[-0.00934828, -0.021600354, 0.07254467, 0.1298...","[-0.02922724, 0.023051806, -0.07529262, -0.079...","[0.0014400505, -0.003789988, -0.0013360691, 0...."
3,3,154126,The Discovery of India,In conjunction with the Jawaharlal Nehru Memor...,Paperback,Jawaharlal Nehru,1.386309,"['History', 'India', 'Nonfiction', 'Politics',...",-0.250787,-0.348547,...,0.013985,1.431631,Non Fiction,"[discovery, india]","[non, fiction]","[jawaharlal, nehru]","[-0.016534291, 0.012368873, -0.04910896, -0.04...","[-0.007913668, -0.021838255, 0.06931542, 0.133...",NaN,"[0.019472577, -0.019621473, -0.0029314258, 0.0..."
4,4,298663,The Killer Inside Me,"Everyone in the small town of Central City, Te...",Paperback,Jim Thompson,-0.484222,"['Fiction', 'Crime', 'Mystery', 'Noir', 'Thril...",0.207299,0.294163,...,0.447853,1.338255,Thriller/Mystery,"[killer, inside]",[thrillermystery],"[jim, thompson]","[-0.025093574, 0.026218837, -0.08757598, -0.08...","[-0.0087274825, 0.0021301615, -0.0008735442, -...","[-0.025739554, 0.024901556, -0.084506966, -0.0...","[0.042012252, -0.031864345, -0.002653681, 0.05..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15451,15565,1038645,The Burning Times,The year is 1357. The Inquisition rages throug...,Paperback,Jeanne Kalogridis,0.381049,"['Historical Fiction', 'Fiction', 'Historical'...",-0.936634,-0.855912,...,-0.616247,0.062579,Historical Fiction,"[burning, time]","[historical, fiction]","[jeanne, kalogridis]","[-0.041203413, 0.051672827, -0.1401742, -0.142...","[-0.00934828, -0.021600354, 0.07254467, 0.1298...","[-0.0035263237, 0.01344164, -0.010550575, -0.0...","[0.09098838, -0.071993135, -0.005877337, 0.143..."
15452,15567,337100,The Runes of the Earth,Beginning ten years after the end of the accla...,Hardcover,Stephen R. Donaldson,0.946065,"['Fantasy', 'Fiction', 'Epic Fantasy', 'Scienc...",-0.298514,-0.463990,...,0.016072,-0.116297,Fantasy,"[rune, earth]",[fantasy],"[stephen, r, donaldson]","[-0.04009235, 0.030851223, -0.09791344, -0.097...","[-0.10737588, 0.01355762, -0.17755514, -0.1037...","[-0.042346727, 0.04779449, -0.13836665, -0.133...","[0.029988524, -0.024467092, 0.0021872513, 0.04..."
15453,15568,9817,Ten Days in the Hills,It is the morning after the Academy Awards. Ma...,Hardcover,Jane Smiley,0.606985,"['Fiction', 'Contemporary', 'Romance', 'Litera...",-0.853062,-0.404413,...,0.296962,-0.248938,Classic Fiction,"[ten, day, hill]","[classic, fiction]","[jane, smi

In [219]:
# recommender = ContentBasedRecommender(df)
# param_grid = {'vector_size': [100, 200], 'min_count': [1, 5], 'epochs': [10, 20]}
# param_grid_word={'vector_size': [100,200], 'window': [1, 5], 'min_count': [1,5], 'workers': [4]}
# best_params, best_score=recommender.tune_word2vec(param_grid_word)

# Get recommendations for the first book
# recommender=ContentBasedRecommender(df,word2vec_params=best_params_word,doc2vec_params=best_params_doc)
# recommendations = recommender.get_recommendations_for_book(book_index=5597, top_n=5)
# print(recommendations)